<a href="https://colab.research.google.com/github/TerriblePepito/TerriblePepitostest/blob/main/export%20de%20pdf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
!pip install pdfplumber
import re
from datetime import datetime
import pdfplumber

In [18]:
from google.colab import files
files.upload()


Saving closedTrades_01.08.25_31.08.25.pdf to closedTrades_01.08.25_31.08.25 (1).pdf


{'closedTrades_01.08.25_31.08.25 (1).pdf': b'%PDF-1.5\n%\xe2\xe3\xcf\xd3\n2 0 obj\n<</ColorSpace/DeviceRGB/Subtype/Image/Height 99/Filter/FlateDecode/Type/XObject/Width 1100/Length 6427/BitsPerComponent 8>>stream\nx\x9c\xed\xddmLT\xd7\xbe\xc7q\xdf\x99\xf8\xa2!11\xc6\x18c\x0c\x81\x9c\xd8\x98\x1a{\xedQ\x8e\xbd\xd1C\xf5\xfa\x90\xd2\xf6V\xad\xa7T\xeb\xd3Q\xbcZ\xad\x0f\xb4z\x95Z\xad\xb5\x16\xcbikQA\x91VEA\xc1*JE\xc0\x07\xec\x13\xd6\x16\x14\xc5\x87"\x8a\xa5\x88\x82Z\x04\x14\x9c\xd93sW\x9d\x9e\xe9\xb8\xd6\xde{\xf6\x0c \\\xf9~2/\xda\x99\xff^\xb3\xf6\x1e^\xac\x9f{\xef\xffv\xb9\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xd0\x

In [20]:
!pip install pdfplumber
import re
from datetime import datetime
import pdfplumber

pdf_path = r"/content/closedTrades_01.08.25_31.08.25 (1).pdf"

with pdfplumber.open(pdf_path) as pdf:
    print("Nombre de pages :", len(pdf.pages))

    page1 = pdf.pages[0]
    text = page1.extract_text()
    print(text[:1000])

TICKET_RE = re.compile(r"^\s*(\d{6,})\s+([#A-Za-z0-9_.()]+)\s+([-–]?\d[\d']*) ")
DATE_RE = re.compile(r"(\d{2}\.\d{2}\.\d{4})")
CHF_RE = re.compile(r"([-–]?\d+(?:[.,]\d+)?)\s*CHF")


def parse_volume(s):
    return int(s.replace("'", "").replace("–", "-"))


def parse_trades(pdf_path):
    trades = []

    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            lines = page.extract_text().splitlines()

            current = []
            for line in lines:
                if TICKET_RE.match(line):
                    if current:
                        trade = parse_trade_block(current)
                        if trade:
                            trades.append(trade)
                    current = [line]
                else:
                    current.append(line)

            if current:
                trade = parse_trade_block(current)
                if trade:
                    trades.append(trade)

    return trades


def parse_trade_block(block):
    text = " ".join(block)

    m = TICKET_RE.match(block[0])
    if not m:
        return None

    instrument = m.group(2)
    volume = parse_volume(m.group(3))

    dates = DATE_RE.findall(text)
    if len(dates) < 2:
        return None

    close_date = datetime.strptime(dates[1], "%d.%m.%Y").date()

    chf_values = CHF_RE.findall(text.replace("–", "-"))
    if not chf_values:
        return None

    pp_chf = float(chf_values[-1].replace(",", "."))

    return {
        "instrument": instrument,
        "volume": volume,
        "date": close_date,
        "pp_chf": pp_chf
    }

Nombre de pages : 13
Transactions fermées
December 13, 2025 16:33:35
Statement August 01, 2025 00:00:00 - September 01, 2025 00:00:00
Ticket Instrument Volume Date Vendu Acheté Pertes & Profits Clé Bp
CHF
226557243 GBPUSD 2'000 13.05.2025 14:00:44 1.32135
SQB-
01.08.2025 09:30:23 1.32059 1.52 USD 1.24 CHF 1650504
226598142 GBPUSD 10'000 01.08.2025 14:37:36 1.32774
13.30 USD 10.75 SQB-
01.08.2025 14:39:52 1.32641 CHF 1650504
226599147 EURUSD 5'000 01.08.2025 14:49:46 1.15480
SQB-
01.08.2025 14:50:53 1.15403 3.85 USD 3.10 CHF 1650504
226599176 USDCHF -5'000 01.08.2025 14:48:21 0.80614
SQB-
01.08.2025 14:51:20 0.80649 1.75 CHF 1.75 CHF 1650504
226599482 EURUSD 5'000 01.08.2025 14:53:38 1.15523
SQB-
01.08.2025 14:56:55 1.15434 4.45 USD 3.59 CHF 1650504
226599494 USDCHF -5'000 01.08.2025 14:54:04 0.80563
SQB-
01.08.2025 14:57:11 0.80629 3.30 CHF 3.30 CHF 1650504
226599761 EURUSD 7'000 01.08.2025 15:01:49 1.15475
SQB-
01.08.2025 15:03:44 1.15430 3.15 USD 2.54 CHF 1650504
226599783 GBPUSD 10'

In [21]:
print("instrument")

instrument
